In [ ]:
using Pkg
Pkg.activate("..")

In [2]:
using JLD2
X_train = load("../data/imdb_dataset_prepared.jld2", "X_train")
y_train = load("../data/imdb_dataset_prepared.jld2", "y_train")
X_test = load("../data/imdb_dataset_prepared.jld2", "X_test")
y_test = load("../data/imdb_dataset_prepared.jld2", "y_test")
embeddings = load("../data/imdb_dataset_prepared.jld2", "embeddings")
vocab = load("../data/imdb_dataset_prepared.jld2", "vocab")
nothing

embedding_dim = size(embeddings,1);

In [3]:
using Flux, Random
Random.seed!(0)

model = Chain(
    Flux.Embedding(length(vocab), embedding_dim),
    Flux.RNN(embedding_dim => 16, relu, return_state = true),
    x -> x[end],
    Flux.flatten,
    Dense(16, 1, σ)
)


Chain(
  Embedding(12849 => 50),               # 642_450 parameters
  RNN(50 => 16, relu),                  # 1_072 parameters
  var"#11#12"(),
  Flux.flatten,
  Dense(16 => 1, σ),                    # 17 parameters
)                   # Total: 6 arrays, 643_539 parameters, 2.455 MiB.

In [4]:
# add Glove embeddings to Embedding layer
model.layers[1].weight .= embeddings;

In [5]:
using Printf, Statistics

function train_loop(print=true)
    dataset = Flux.DataLoader((X_train, y_train), batchsize=128, shuffle=true)

    loss(m, x, y) = Flux.Losses.binarycrossentropy(m(x), y)
    accuracy(m, x, y) = mean((m(x) .> 0.5) .== (y .> 0.5))

    opt = Optimisers.setup(RMSProp(), model)

    epochs = 12
    for epoch in 1:epochs
        total_loss = 0.0
        total_acc = 0.0
        num_samples = 0

        t = @elapsed begin
            for (x, y) in dataset
                Flux.reset!(model)
                grads = Flux.gradient(model) do m
                loss(m, x, y)
                end
                Optimisers.update!(opt, model, grads[1])
                total_loss += loss(model, x, y)
                total_acc += accuracy(model, x, y)
                num_samples += 1
            end

            train_loss = total_loss / num_samples
            train_acc = total_acc / num_samples

            test_acc = accuracy(model, X_test, y_test)
            test_loss = loss(model, X_test, y_test)
        end

        if print
            println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)", 
                epoch, t, train_loss, train_acc, test_loss, test_acc))
        end
    end
end

train_loop (generic function with 2 methods)

In [6]:
train_loop()

Epoch: 1 (63.43s) 	Train: (l: 0.69, a: 0.52) 	Test: (l: 0.69, a: 0.50)
Epoch: 2 (17.56s) 	Train: (l: 0.68, a: 0.54) 	Test: (l: 0.69, a: 0.51)
Epoch: 3 (14.69s) 	Train: (l: 0.58, a: 0.70) 	Test: (l: 0.53, a: 0.75)
Epoch: 4 (14.01s) 	Train: (l: 0.47, a: 0.79) 	Test: (l: 0.53, a: 0.79)
Epoch: 5 (14.61s) 	Train: (l: 0.41, a: 0.83) 	Test: (l: 0.44, a: 0.82)
Epoch: 6 (13.98s) 	Train: (l: 0.37, a: 0.85) 	Test: (l: 0.39, a: 0.84)
Epoch: 7 (14.08s) 	Train: (l: 0.33, a: 0.87) 	Test: (l: 0.45, a: 0.83)
Epoch: 8 (14.47s) 	Train: (l: 0.31, a: 0.88) 	Test: (l: 0.39, a: 0.84)
Epoch: 9 (14.55s) 	Train: (l: 0.28, a: 0.90) 	Test: (l: 0.36, a: 0.85)
Epoch: 10 (14.04s) 	Train: (l: 0.26, a: 0.90) 	Test: (l: 0.39, a: 0.85)
Epoch: 11 (15.26s) 	Train: (l: 0.24, a: 0.91) 	Test: (l: 0.46, a: 0.85)
Epoch: 12 (13.88s) 	Train: (l: 0.23, a: 0.92) 	Test: (l: 0.36, a: 0.87)


In [7]:
using BenchmarkTools

Random.seed!(0)

model = Chain(
    Flux.Embedding(length(vocab), embedding_dim),
    Flux.RNN(embedding_dim => 16, relu, return_state = true),
    x -> x[end],
    Flux.flatten,
    Dense(16, 1, σ)
)

model.layers[1].weight .= embeddings;

@benchmark train_loop(false)

BenchmarkTools.Trial: 1 sample with 1 evaluation per sample.
 Single result which took 178.951 s (14.64% GC) to evaluate,
 with a memory estimate of 154.19 GiB, over 158204593 allocations.